In [1]:
from IPython.display import HTML, display
css_file_path = "../../design/custom.css"
styles = "<link rel='stylesheet' href='%s'>" % css_file_path
display(HTML(styles))

# Tema 1.1 $\quad$ Descarga de datos

En este primer notebook vamos a aprender cómo descargar los datos para poder trabajar con ellos. Vamos a utilizar las siguientes herramientas:

* Yahoo Finance
* Quandl
* AlphaVantage

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [3]:
# print(plt.style.available) #list of available styles
plt.style.use('ggplot')

warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Descargar datos de Yahoo Finance

`yfinance` es una librería de Python que proporciona una interfaz sencilla para acceder y descargar datos financieros de Yahoo Finance. Esta biblioteca facilita la obtención de información detallada sobre precios históricos, datos de acciones, divisas, bonos, índices, etc. Esta librería es directamente compatible con `pandas`, ya que devuelve los datos en formato DataFrame.

Podéis ver la información sobre la librería en [yfinance](https://pypi.org/project/yfinance/)

Para instalar la libería si fuese necesario:

```pip install yfinance```

In [4]:
import yfinance as yf

In [5]:
# Descargamos los datos de Apple ('AAPL') entre los años 2000 y 2010

data_yahoo = yf.download('AAPL',
                         start='2000-01-01', # Se indica con el argumento 'start' una fecha de incio (incluida)
                         end='2011-01-01')   # Se indica con el argumento 'end' una fecha fin (excluida)

[*********************100%%**********************]  1 of 1 completed


La variable `data_yahoo` es un DataFrame de Pandas con la siguiente información:

* Como índice del DataFrame, **Date**: La fecha a la que se recoge la información
* **Open**: Precio de apertura del activo
* **High**: Precio máximo alcanzado por el activo
* **Low**: Precio mínimo alcanzado por el activo
* **Close**: Precio de cierre del activo
* **Adj Close**: Precio de cierre ajustado por eventos corporativos, como dividendos, splits, etc.
* **Volume**: Valor de las transacciones negociadas

Veamos el aspecto que tiene:

In [8]:
data_yahoo.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442,0.846127,535796800
2000-01-04,0.966518,0.987723,0.903460,0.915179,0.774790,512377600
2000-01-05,0.926339,0.987165,0.919643,0.928571,0.786128,778321600
2000-01-06,0.947545,0.955357,0.848214,0.848214,0.718098,767972800
2000-01-07,0.861607,0.901786,0.852679,0.888393,0.752113,460734400


In [ ]:
data_yahoo.tail() # Incluye el 31 de diciembre de 2010

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-12-27,11.530357,11.622857,11.482857,11.595714,9.816923,249816000
2010-12-28,11.639643,11.666429,11.609286,11.623929,9.840810,175924000
2010-12-29,11.650714,11.658929,11.610714,11.617500,9.835371,163139200
2010-12-30,11.624286,11.625357,11.537500,11.559286,9.786086,157494400
2010-12-31,11.533929,11.552857,11.475357,11.520000,9.752829,193508000


In [ ]:
data_yahoo.shape # 2767 filas, 6 columnas

(2767, 6)

Lo más común es trabajar con los precios de cierre. Concretamente, vamos a seleccionar la columna de precios ajustados `Adj Close`.

**OBSERVACIÓN:** Hay que tener cuidado con las transformaciones que hacemos sobre el DataFrame. Ojo si lo que seleccionamos es un DataFrame o una Serie.

In [10]:
# Si elijo una sola columna, me devuelve una serie
serie_yahoo = data_yahoo['Adj Close']
# print(type(serie_yahoo))
serie_yahoo.head()


<class 'pandas.core.series.Series'>


Date
2000-01-03    0.846127
2000-01-04    0.774790
2000-01-05    0.786128
2000-01-06    0.718098
2000-01-07    0.752113
Name: Adj Close, dtype: float64

In [ ]:
# Si elijo una lista de columnas (aunque solo haya una), me devuelve un dataframe con una sola columna
df1_yahoo = data_yahoo[['Adj Close']]
df1_yahoo.head()

,Adj Close
Date,
2000-01-03,0.846127
2000-01-04,0.774790
2000-01-05,0.786128
2000-01-06,0.718098
2000-01-07,0.752113


En la celda a continuación, descargamos los datos de varios activos financieros a la vez. Simplemente pasamos a `yf.download` una lista de activos en lugar de solo uno:

In [13]:
activos = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'TSLA', 'NVDA', 'ADBE', 'NFLX']
df_yahoo = yf.download(activos,
                       start='2000-01-01',
                       end='2011-01-01')

[*********************100%%**********************]  8 of 8 completed


En este caso, al descargar una lista de activos, yfinance nos devuelve un DataFrame multinivel en las columnas `(atributo, activo)`, donde el primer nivel representa los atributos (open, high, low, close, etc.) y el segundo nivel el activo correspondiente (AAPL, MSFT, etc.):


In [14]:
df_yahoo.head()

Price      Adj Close                                                           \
Ticker          AAPL       ADBE      AMZN GOOG       MSFT NFLX      NVDA TSLA   
Date                                                                            
2000-01-03  0.846127  16.274672  4.468750  NaN  36.065567  NaN  0.894608  NaN   
2000-01-04  0.774790  14.909399  4.096875  NaN  34.847267  NaN  0.870721  NaN   
2000-01-05  0.786128  15.204173  3.487500  NaN  35.214699  NaN  0.842055  NaN   
2000-01-06  0.718098  15.328288  3.278125  NaN  34.035069  NaN  0.787112  NaN   
2000-01-07  0.752113  16.072983  3.478125  NaN  34.479847  NaN  0.800251  NaN   

Price          Close             ...      Open          Volume            \
Ticker          AAPL       ADBE  ...      NVDA TSLA       AAPL      ADBE   
Date                             ...                                       
2000-01-03  0.999442  16.390625  ...  0.984375  NaN  535796800   7384400   
2000-01-04  0.915179  15.015625  ...  0.958333  NaN  512377600   7813200   
2000-01-05  0.928571  15.312500  ...  0.921875  NaN  778321600  14927200   
2000-01-06  0.848214  15.437500  ...  0.917969  NaN  767972800  10221200   
2000-01-07  0.888393  16.187500  ...  0.854167  NaN  460734400   8253200   

Price                                                     
Ticker           AMZN GOOG      MSFT NFLX      NVDA TSLA  
Date                                                      
2000-01-03  322352000  NaN  53228400  NaN  30091200  NaN  
2000-01-04  349748000  NaN  54119000  NaN  30048000  NaN  
2000-01-05  769148000  NaN  64059600  NaN  18835200  NaN  
2000-01-06  375040000  NaN  54976600  NaN  12048000  NaN  
2000-01-07  210108000  NaN  62013600  NaN   7118400  NaN  

[5 rows x 48 columns]

Pongamos las tres situaciones posibles como ejercicio:

1. Del DataFrame descargado, muestra el `Open` de todos los activos.
2. Del DataFrame descargado, muestra el `Close` de `'MSFT'`.
3. Del DataFrame descargado, muestra todos los atributos de `'NVDA'`.

In [15]:
# 1. Del DataFrame descargado, muestra el `Open` de todos los activos.

# SOLUCION
solucion_1 = df_yahoo['Open']
solucion_1.head()

Ticker,AAPL,ADBE,AMZN,GOOG,MSFT,NFLX,NVDA,TSLA
Date,,,,,,,,
2000-01-03,0.936384,16.812500,4.075000,NaN,58.68750,NaN,0.984375,NaN
2000-01-04,0.966518,15.750000,4.268750,NaN,56.78125,NaN,0.958333,NaN
2000-01-05,0.926339,14.562500,3.525000,NaN,55.56250,NaN,0.921875,NaN
2000-01-06,0.947545,15.359375,3.565625,NaN,56.09375,NaN,0.917969,NaN
2000-01-07,0.861607,15.390625,3.350000,NaN,54.31250,NaN,0.854167,NaN


In [16]:
# 2. Del DataFrame descargado, muestra el `Close` de 'MSFT'.

# SOLUCION
solucion_2 = df_yahoo['Close', 'MSFT']
solucion_2.head()

Date
2000-01-03    58.28125
2000-01-04    56.31250
2000-01-05    56.90625
2000-01-06    55.00000
2000-01-07    55.71875
Name: (Close, MSFT), dtype: float64

In [17]:
# 3. Del DataFrame descargado, muestra todos los atributos de 'NVDA'.

# SOLUCION
L = [x for x in df_yahoo.columns if x[1] == 'NVDA']
solucion_3 = df_yahoo[L]
solucion_3.columns = solucion_3.columns.droplevel(1) # Prueba a comentar esta línea. ¿Qué sucede?
solucion_3.head()

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2000-01-03,0.894608,0.975260,0.992188,0.919271,0.984375,30091200
2000-01-04,0.870721,0.949219,0.960938,0.901042,0.958333,30048000
2000-01-05,0.842055,0.917969,0.937500,0.904948,0.921875,18835200
2000-01-06,0.787112,0.858073,0.917969,0.822917,0.917969,12048000
2000-01-07,0.800251,0.872396,0.881510,0.841146,0.854167,7118400


## 2. Descargarse datos de Quandl

`quandl` es una plataforma que proporciona acceso a una amplia gama de datos financieros y económicos de diferentes fuentes. La librería Quandl para Python ofrece una interfaz sencilla para acceder a estos datos desde la plataforma. Al igual que `yfinance`, esta librería es directamente compatible con `pandas`, ya que devuelve los datos en formato DataFrame.

Para instalar las liberías si fuese necesario:

```
pip install quandl

```

In [ ]:
import quandl as ql

In [ ]:
# Descargamos los datos de Apple ('WIKI/AAPL') entre los años 2000 y 2010

df_quandl = ql.get(dataset='WIKI/AAPL',
                   start_date='2000-01-01', # Se indica con el argumento 'start_date' una fecha de incio (incluida)
                   end_date='2010-12-31') # Se indica con el argumento 'end_date' una fecha fin (OJO! incluida!)

La variable `df_quandl` es un DataFrame de Pandas con la siguiente información:

* Como índice del DataFrame, **Date**: La fecha a la que se recoge la información
* **Open**: Precio de apertura del activo
* **High**: Precio máximo alcanzado por el activo
* **Low**: Precio mínimo alcanzado por el activo
* **Close**: Precio de cierre del activo
* **Volume**: Valor de las transacciones negociadas
* **Ex-Dividend**: Periodo de negociación sin derecho al próximo dividendo
* **Split Ratio**: Proporción en la que se dividen las acciones
* **Adj. Open**: Precio de apertura ajustado por eventos corporativos, como dividendos, splits, etc.
* **Adj. High**: Precio máximo ajustado por eventos corporativos, como dividendos, splits, etc.
* **Adj. Low**: Precio mínimo ajustado por eventos corporativos, como dividendos, splits, etc.
* **Adj. Close**: Precio de cierre ajustado por eventos corporativos, como dividendos, splits, etc.
* **Adj. Volume**: Valor de las transacciones negociadas ajustado

Veamos el aspecto que tiene:

In [ ]:
df_quandl.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2000-01-03,104.87,112.50,101.69,111.94,4783900.0,0.0,1.0,3.369314,3.614454,3.267146,3.596463,133949200.0
2000-01-04,108.25,110.62,101.19,102.50,4574800.0,0.0,1.0,3.477908,3.554053,3.251081,3.293170,128094400.0
2000-01-05,103.75,110.56,103.00,104.00,6949300.0,0.0,1.0,3.333330,3.552125,3.309234,3.341362,194580400.0
2000-01-06,106.12,107.00,95.00,95.00,6856900.0,0.0,1.0,3.409475,3.437748,3.052206,3.052206,191993200.0
2000-01-07,96.50,101.00,95.50,99.50,4113700.0,0.0,1.0,3.100399,3.244977,3.068270,3.196784,115183600.0


In [ ]:
df_quandl.tail()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2010-12-27,322.8519,325.44,321.52,324.68,8922000.0,0.0,1.0,41.490969,41.823576,41.319801,41.725905,62454000.0
2010-12-28,325.9102,326.66,325.06,325.47,6283000.0,0.0,1.0,41.884003,41.980363,41.774741,41.827431,43981000.0
2010-12-29,326.2200,326.45,325.10,325.29,5826400.0,0.0,1.0,41.923817,41.953375,41.779881,41.804299,40784800.0
2010-12-30,325.4800,325.51,323.05,323.66,5624800.0,0.0,1.0,41.828716,41.832572,41.516428,41.594821,39373600.0
2010-12-31,322.9500,323.48,321.31,322.56,6911000.0,0.0,1.0,41.503576,41.571689,41.292813,41.453456,48377000.0


In [ ]:
df_quandl.shape # 2767 filas, 12 columnas

(2767, 12)

A partir de aquí, la metodología de trabajo es equivalente a trabajar con los datos de `yfinance` como en el apartado anterior. Sin embargo, hay una diferencia respecto a descargar un conjunto de activos:

In [ ]:
# Descargamos los datos de Apple ('WIKI/AAPL') y Microsoft ('WIKI/MSFT') entre los años 2000 y 2010

activos = ["WIKI/AAPL", "WIKI/MSFT"]

df_quandl = ql.get(dataset=activos,
                   start_date='2000-01-01', # Se indica con el argumento 'start_date' una fecha de incio (incluida)
                   end_date='2010-12-31') # Se indica con el argumento 'end_date' una fecha fin (OJO! incluida!)

Aunque la descarga de los datos es equivalente a `yfinance`, en este caso no se genera un DataFrame multinivel en las columnas como antes. Con la librería Quandl, se define una columna por cada `activo - atributo`, veámoslo:

In [ ]:
df_quandl.head()

,WIKI/AAPL - Open,WIKI/AAPL - High,WIKI/AAPL - Low,WIKI/AAPL - Close,WIKI/AAPL - Volume,WIKI/AAPL - Ex-Dividend,WIKI/AAPL - Split Ratio,WIKI/AAPL - Adj. Open,WIKI/AAPL - Adj. High,WIKI/AAPL - Adj. Low,...,WIKI/MSFT - Low,WIKI/MSFT - Close,WIKI/MSFT - Volume,WIKI/MSFT - Ex-Dividend,WIKI/MSFT - Split Ratio,WIKI/MSFT - Adj. Open,WIKI/MSFT - Adj. High,WIKI/MSFT - Adj. Low,WIKI/MSFT - Adj. Close,WIKI/MSFT - Adj. Volume
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,104.87,112.50,101.69,111.94,4783900.0,0.0,1.0,3.369314,3.614454,3.267146,...,112.00,116.56,26614200.0,0.0,1.0,39.066089,39.482146,37.278708,38.796484,53228400.0
2000-01-04,108.25,110.62,101.19,102.50,4574800.0,0.0,1.0,3.477908,3.554053,3.251081,...,112.25,112.62,27059500.0,0.0,1.0,37.797947,38.982878,37.361920,37.485073,54119000.0
2000-01-05,103.75,110.56,103.00,104.00,6949300.0,0.0,1.0,3.333330,3.552125,3.309234,...,109.37,113.81,32029800.0,0.0,1.0,36.985804,38.733244,36.403324,37.881159,64059600.0
2000-01-06,106.12,107.00,95.00,95.00,6856900.0,0.0,1.0,3.409475,3.437748,3.052206,...,108.37,110.00,27488300.0,0.0,1.0,37.341949,37.901130,36.070479,36.613017,54976600.0
2000-01-07,96.50,101.00,95.50,99.50,4113700.0,0.0,1.0,3.100399,3.244977,3.068270,...,107.31,111.44,31006800.0,0.0,1.0,36.153690,37.361920,35.717662,37.092315,62013600.0


Pongamos las tres situaciones posibles como ejercicio:

1. Del DataFrame descargado, muestra el `Open` de todos los activos.
2. Del DataFrame descargado, muestra el `Close` de `'MSFT'`.
3. Del DataFrame descargado, muestra todos los atributos de `'AAPL'`.

In [ ]:
# 1. Del DataFrame descargado, muestra el `Open` de todos los activos.

# SOLUCION
L = [x for x in df_quandl.columns if "Open" in x and "Adj." not in x]
solucion_1 = df_quandl[L]
solucion_1.head()

,WIKI/AAPL - Open,WIKI/MSFT - Open
Date,,
2000-01-03,104.87,117.37
2000-01-04,108.25,113.56
2000-01-05,103.75,111.12
2000-01-06,106.12,112.19
2000-01-07,96.50,108.62


In [ ]:
# 2. Del DataFrame descargado, muestra el `Close` de 'MSFT'.

# SOLUCION
solucion_2 = df_quandl["WIKI/MSFT - Close"]
solucion_2.head()

Date
2000-01-03    116.56
2000-01-04    112.62
2000-01-05    113.81
2000-01-06    110.00
2000-01-07    111.44
Name: WIKI/MSFT - Close, dtype: float64

In [ ]:
# 3. Del DataFrame descargado, muestra todos los atributos de 'AAPL'.

# SOLUCION
L = [x for x in df_quandl.columns if "AAPL" in x]
solucion_3 = df_quandl[L]
solucion_3.head()

,WIKI/AAPL - Open,WIKI/AAPL - High,WIKI/AAPL - Low,WIKI/AAPL - Close,WIKI/AAPL - Volume,WIKI/AAPL - Ex-Dividend,WIKI/AAPL - Split Ratio,WIKI/AAPL - Adj. Open,WIKI/AAPL - Adj. High,WIKI/AAPL - Adj. Low,WIKI/AAPL - Adj. Close,WIKI/AAPL - Adj. Volume
Date,,,,,,,,,,,,
2000-01-03,104.87,112.50,101.69,111.94,4783900.0,0.0,1.0,3.369314,3.614454,3.267146,3.596463,133949200.0
2000-01-04,108.25,110.62,101.19,102.50,4574800.0,0.0,1.0,3.477908,3.554053,3.251081,3.293170,128094400.0
2000-01-05,103.75,110.56,103.00,104.00,6949300.0,0.0,1.0,3.333330,3.552125,3.309234,3.341362,194580400.0
2000-01-06,106.12,107.00,95.00,95.00,6856900.0,0.0,1.0,3.409475,3.437748,3.052206,3.052206,191993200.0
2000-01-07,96.50,101.00,95.50,99.50,4113700.0,0.0,1.0,3.100399,3.244977,3.068270,3.196784,115183600.0


## 3. Descargarse datos de AlphaVantage

AlphaVantage es una plataforma que proporciona datos financieros y servicios relacionados con el mercado. En lugar de utilizar un paquete de python, esta herramienta ofrece una **API** (Interfaz de Programación de Aplicaciones) que permite a los desarrolladores acceder mediante el protocolo HTTP a los datos, sea cual sea el lenguaje de programación.

Una **API** es una herramienta de comunicación entre diferentes aplicaciones, normalmente mediante servicios web. Se basa en la transferencia de información entre cliente y servidor. Utiliza el protocolo HTTP estándar (GET, POST, PUT, DELETE) para realizar acciones como leer, crear, actualizar y eliminar recursos.

En este caso, la API de AlphaVantage nos proporciona acceso a datos financieros y servicios relacionados con el mercado. Para poder lanzar peticiones HTTP con Python a la API necesitamos la librería `requests`.

In [ ]:
import requests

Para lanzar consultas a la API de AlphaVantage necesitamos una `key`. Sin embargo, es un servicio gratuito que podéis conseguir en menos de 1 minuto pinchando [aquí](https://www.alphavantage.co/support/#api-key). La web os genera una clave de acceso con el aspecto siguiente:

*ABCDEFGHIJKLMNOP*

Lo ideal es que guardéis esa clave en vuestro ordenador personal, en un fichero seguro. Entonces, para recuperar la clave, podemos ejecutar el siguiente código:

In [ ]:
# Specify the path to the file containing your API key
api_key_file = '../../credentials/alphavantage_test.txt'

# Try to open the file and read the API key
try:
    with open(api_key_file, 'r') as file:
        alpha_key = file.read().strip()  # Read the content and remove leading/trailing whitespace

    # Check if the API key is not empty
    if alpha_key:
        print("Alpha Key: Ha funcionado!")
        print("      Key:", alpha_key)
    else:
        print("The API key file is empty. Please check the file content.")

except FileNotFoundError:
    print(f"The file '{api_key_file}' was not found. Please make sure the file exists.")
    alpha_key = "ABCDEFGHIJKLMNOP"
except Exception as e:
    print(f"An error occurred while reading the API key: {str(e)}")


The file '../../credentials/alphavantage_test.txt' was not found. Please make sure the file exists.


Como la mayoría de APIs REST, para realizar una petición tenemos que preparar la URL de forma correcta para hacer la consulta. Veamos este paso poco a poco:

**Preparación de la URL para consultar a la API:**

Lo primero que necesitamos es acceder a la documentación de la API (https://www.alphavantage.co/documentation/) por si necesitamos algún parámetro adicional. Concretamente, vamos a utilizar la función **TIME_SERIES_INTRADAY** para descargar los datos de **Apple** con un **intervalo de 5 minutos**. Para ello definimos por trozos la URL:

In [ ]:
function = "function=TIME_SERIES_INTRADAY"
symbol = "symbol=AAPL"
interval = "interval=5min"
key = "apikey=" + alpha_key # Aquí añadimos nuestra clave

Ahora tenemos que generar la URL completa:

In [ ]:
URL = "https://www.alphavantage.co/query?"
URL = URL + '&'.join([function, symbol, interval, key])
print(URL)

https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=5min&apikey=ABCDEFGHIJKLMNOP


¿Habéis probado a entrar en el enlace de arriba? Así es como muestra el explorador (Chrome, Firefox, o el que utilicéis) los datos **JSON**, que es como la API de Alphavantage devuelve la información solicitada.

**Consulta a la API usando la URL:**

Ahora vamos a hacer la petición con la librería `requests`:

In [ ]:
data_5min = requests.get(URL).json()

print(data_5min)

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume', '2. Symbol': 'AAPL', '3. Last Refreshed': '2024-02-23 19:55:00', '4. Interval': '5min', '5. Output Size': 'Compact', '6. Time Zone': 'US/Eastern'}, 'Time Series (5min)': {'2024-02-23 19:55:00': {'1. open': '182.3500', '2. high': '182.3500', '3. low': '182.2300', '4. close': '182.2400', '5. volume': '6326'}, '2024-02-23 19:50:00': {'1. open': '182.2900', '2. high': '182.3800', '3. low': '182.2900', '4. close': '182.3200', '5. volume': '2030'}, '2024-02-23 19:45:00': {'1. open': '182.2900', '2. high': '182.3000', '3. low': '182.2800', '4. close': '182.2850', '5. volume': '1488'}, '2024-02-23 19:40:00': {'1. open': '182.2900', '2. high': '182.3000', '3. low': '182.2800', '4. close': '182.2850', '5. volume': '761'}, '2024-02-23 19:35:00': {'1. open': '182.2450', '2. high': '182.3000', '3. low': '182.2450', '4. close': '182.3000', '5. volume': '2845'}, '2024-02-23 19:30:00': {'1. open': '182.2500', 

Por supuesto, los datos están en formato JSON, por lo que es difícil interpretarlos. Vamos a transformar el JSON en un DataFrame de Pandas:

In [ ]:
# Convierte el JSON en un DataFrame de Pandas
df_5min = pd.DataFrame.from_dict(data_5min['Time Series (5min)'], orient='index')

# Cambia los nombres de las columnas para eliminar los números y los puntos
df_5min.columns = [col.split('. ')[1] for col in df_5min.columns] # Prueba a comentar esta línea

df_5min.head()

,open,high,low,close,volume
2024-02-23 19:55:00,182.3500,182.3500,182.2300,182.2400,6326
2024-02-23 19:50:00,182.2900,182.3800,182.2900,182.3200,2030
2024-02-23 19:45:00,182.2900,182.3000,182.2800,182.2850,1488
2024-02-23 19:40:00,182.2900,182.3000,182.2800,182.2850,761
2024-02-23 19:35:00,182.2450,182.3000,182.2450,182.3000,2845


A partir de aquí, podemos trabajar normalmente con los datos como hemos hecho con `yfinance` y `quandl`.

Hagamos ahora otro ejemplo para obtener los datos diarios. Simplemente llamamos a la función **TIME_SERIES_DAILY**:

In [ ]:
function = "function=TIME_SERIES_DAILY"
symbol = "symbol=AAPL"
key = "apikey=" + alpha_key

URL = "https://www.alphavantage.co/query?"
URL = URL + '&'.join([function, symbol, key])
print(URL)

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AAPL&apikey=ABCDEFGHIJKLMNOP


In [ ]:
data_daily = requests.get(URL).json()
df_daily = pd.DataFrame.from_dict(data_daily['Time Series (Daily)'], orient='index')
df_daily.columns = [col.split('. ')[1] for col in df_daily.columns]
df_daily.head()

,open,high,low,close,volume
2024-02-23,185.0100,185.0400,182.2300,182.5200,45119677
2024-02-22,183.4800,184.9550,182.4600,184.3700,52292208
2024-02-21,181.9400,182.8888,180.6600,182.3200,41529674
2024-02-20,181.7900,182.4300,180.0000,181.5600,53665553
2024-02-16,183.4200,184.8500,181.6650,182.3100,49752465


Hagamos un último ejemplo para obtener información sobre Apple llamando a la función **SYMBOL_SEARCH**:

In [ ]:
function = "function=SYMBOL_SEARCH"
symbol = "keywords=Apple"
key = "apikey=" + alpha_key

URL = "https://www.alphavantage.co/query?"
URL = URL + '&'.join([function, symbol, key])
print(URL)

https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=Apple&apikey=ABCDEFGHIJKLMNOP


In [ ]:
data = requests.get(URL).json()

df_info = pd.DataFrame.from_dict(data['bestMatches'])
df_info.columns = [col.split('. ')[1] for col in df_info.columns]
df_info

,symbol,name,type,region,marketOpen,marketClose,timezone,currency,matchScore
0,APLE,Apple Hospitality REIT Inc,Equity,United States,09:30,16:00,UTC-04,USD,0.8889
1,AAPL,Apple Inc,Equity,United States,09:30,16:00,UTC-04,USD,0.7143
2,AAPL34.SAO,Apple Inc,Equity,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,0.7143
3,APC.DEX,Apple Inc,Equity,XETRA,08:00,20:00,UTC+02,EUR,0.7143
4,APC.FRK,Apple Inc,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.7143
5,AGPL,Apple Green Holding Inc,Equity,United States,09:30,16:00,UTC-04,USD,0.6667
6,0R2V.LON,Apple Inc.,Equity,United Kingdom,08:00,16:30,UTC+01,USD,0.6667
7,500014.BSE,Apple Finance Limited,Equity,India/Bombay,09:15,15:30,UTC+5.5,INR,0.3846
8,48T.FRK,APPLE HOSPITALITY REIT,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.3704
9,603020.SHH,Apple Flavor Fragrance Group Company Ltd,Equity,Shanghai,09:30,15:00,UTC+08,CNY,0.2222
